In [33]:
#Project: Image data augmentation for a CNN 

In [34]:
import numpy as np
import matplotlib.pyplot as plt
import random
import os
import PIL

import skimage.transform
import skimage.util

In [35]:
### Settings

# Location of dataset and output folder
DATASET_PATH = r"..\..\..\..\..\..\content\dataset"
OUT_PATH = r"..\..\..\..\..\..\content\output"
# OUT_ZIP = "augmented_dataset.zip" (To implement in Windows 10)

# File format to use for new dataset
IMG_EXT = ".png"

# You are welcome to change the seed to get different augmentation effects
SEED = 42
random.seed(SEED)

In [36]:
### Create output directory
try:
  os.makedirs(OUT_PATH)
except FileExistsError:
  print("WARNING: Output directory already exists. Check to make sure it is empty.")

In [37]:
### Function to create 3 new flipped images of the input

def create_flipped(img):

  # Create a list of flipped images
  flipped = []
  flipped.append(np.fliplr(img))
  flipped.append(np.flipud(img))
  flipped.append(np.flipud(np.fliplr(img)))

  return flipped

In [38]:
### Functions that create transforms of the images

# Functions must return one or more 2D Numpy arrays corresponding to the transforms. 
# The array(s) must be stored in a list, even if it’s just one array (e.g. [transformed_img])

def create_translations(img, n):
  translations = []

  # Roll function to roll array elements along a given axis: 
  translations.append(np.roll(img,n,axis=0))
  translations.append(np.roll(np.roll(img,n,axis=1),n,axis=0))

  return translations
    

In [39]:
### Function to open image and create a list of new transforms

def create_transforms(file_path):

  # Open the image
  img = PIL.Image.open(file_path)

  # Convert the image to a Numpy array (keep all color channels)
  img_array = np.asarray(img)

  # Add original image to front of list
  img_tfs = []
  img_tfs.append([img_array])

  # Perform transforms (call above functions)
  img_tfs.append(create_flipped(img_array))

  # Preforms translation
  img_tfs.append(create_translations(img_array, 20))

  # Flatten list of lists (to create one long list of images)
  img_tfs = [img for img_list in img_tfs for img in img_list]

  return img_tfs

In [40]:
### Utility code: Three for-loops to load all images, create transforms, and save in an output directory

# Find the directories in the dataset folder (skip the Jupyter Notebook checkpoints hidden folder)

for label in os.listdir(DATASET_PATH):
  class_dir = os.path.join(DATASET_PATH, label)
  if os.path.isdir(class_dir) and label != ".ipynb_checkpoints":

    # Create output directory
    out_path = os.path.join(OUT_PATH, label)
    os.makedirs(out_path, exist_ok=True)

    # Go through each image in the subfolder
    for i, filename in enumerate(os.listdir(class_dir)):

      # Skip the Jupyter Notebook checkpoints folder that sometimes gets added
      if filename != ".ipynb_checkpoints":

        # Get the root of the filename before the extension
        file_root = os.path.splitext(filename)[0]

        # Do all transforms for that one image
        file_path = os.path.join(DATASET_PATH, label, filename)
        img_tfs = create_transforms(file_path)

        # Save images to new files in output directory
        for i, img in enumerate(img_tfs):

          # Create a Pillow image from the Numpy array
          img_pil = PIL.Image.fromarray(img)

          # Construct filename (<orignal>_<transform_num>.<EXT>)
          out_file_path = os.path.join(out_path, file_root + "_" + str(i) + IMG_EXT)

          # Convert Numpy array to image and save as a file
          img_pil = PIL.Image.fromarray(img)
          img_pil.save(out_file_path)